In [1]:
# imports
import json
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import cv2
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import data as tf_data
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, log_loss
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from typing import List, Dict, Any, Union

from utils import print_progress_bar

print("Modules imported")

Modules imported


In [20]:
# variables
all_labels = ['nature', 'country', 'city']
path = "../datasets/all_data/entropy_results.json"
parallel_jobs = 5

# hyperparameters
test_part = 0.1
epochs = 20
batch_size = 256
learning_rate = 0.005

# check gpu
devices = tf.config.list_physical_devices()
print("Available devices:")
for device in devices:
    print(device.name)
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"The model will run on GPU: {physical_devices[0].name}")
else:
    print("No GPU found, the model will run on CPU.")

Available devices:
/physical_device:CPU:0
/physical_device:GPU:0
The model will run on GPU: /physical_device:GPU:0


In [3]:
# heads
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
                self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.wq = tf.keras.layers.Dense(self.head_dim)
        self.wk = tf.keras.layers.Dense(self.head_dim)
        self.wv = tf.keras.layers.Dense(self.head_dim)

    def call(self, inputs):
        Q = self.wq(inputs)
        K = self.wk(inputs)
        V = self.wv(inputs)

        matmul_qk = tf.matmul(Q, K, transpose_b=True)

        depth = tf.cast(tf.shape(K)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)

        attention_weights = tf.nn.softmax(logits, axis=-1)

        output = tf.matmul(attention_weights, V)
        return output

In [28]:
# model
class EntropyClassifier(tf.keras.Model):
    def __init__(self, possible_labels, folder, debug=False):
        super(EntropyClassifier, self).__init__()
        dwt_output_size = 10
        lvl0_output_size = 17
        lvl1_output_size = 17
        lvl2_output_size = 17
        lvl3_output_size = 153

        embed_size = dwt_output_size + lvl0_output_size + lvl1_output_size + lvl2_output_size + lvl3_output_size  # = 214

        heads = 1  # Choose based on your specific requirements or experimentation
        assert embed_size % heads == 0, "Embedding size needs to be divisible by heads"

        self.possible_labels = possible_labels
        self.debug = debug
        self.folder = folder

        self.dwt_input_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(10),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.5)
        ])
        self.lvl0_input_layer = tf.keras.Sequential([
            tf.keras.layers.Dense(17),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.5)
        ])
        self.lvl1_input_layers = [tf.keras.Sequential([
            tf.keras.layers.Conv2D(1, (2, 2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dropout(0.5)
        ]) for _ in range(17)]
        self.lvl2_input_layers = [tf.keras.Sequential([
            tf.keras.layers.Conv2D(1, (2, 2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            tf.keras.layers.Dropout(0.5)
        ]) for _ in range(17)]
        self.lvl3_input_layers = [tf.keras.Sequential([
            tf.keras.layers.Conv2D(1, (2, 2)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            tf.keras.layers.Dropout(0.5)
        ]) for _ in range(17)]

        # Commenting out the original self-attention layer
        # self.self_attention = SelfAttention(embed_size, heads)

        # Replacing self-attention with dense layers
        self.final_nn = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(len(possible_labels), activation='softmax')
        ])

    def call(self, inputs, training=False):
        lvl0_inputs, lvl1_inputs, lvl2_inputs, lvl3_inputs, dwt_inputs = inputs

        batch_size = tf.shape(dwt_inputs)[0]

        # Ensure inputs have a batch dimension
        if len(lvl1_inputs.shape) == 3:
            lvl1_inputs = tf.expand_dims(lvl1_inputs, axis=0)
        if len(lvl2_inputs.shape) == 3:
            lvl2_inputs = tf.expand_dims(lvl2_inputs, axis=0)
        if len(lvl3_inputs.shape) == 3:
            lvl3_inputs = tf.expand_dims(lvl3_inputs, axis=0)

        dwt_output = self.dwt_input_layer(dwt_inputs)
        lvl0_output = self.lvl0_input_layer(lvl0_inputs)

        lvl1_output = tf.concat([self.lvl1_input_layers[i](lvl1_inputs[:, :, :, i:i + 1]) for i in range(17)], axis=-1)
        lvl2_output = tf.concat([self.lvl2_input_layers[i](lvl2_inputs[:, :, :, i:i + 1]) for i in range(17)], axis=-1)
        lvl3_output = tf.concat([self.lvl3_input_layers[i](lvl3_inputs[:, :, :, i:i + 1]) for i in range(17)], axis=-1)

        concatenated_output = tf.concat([tf.reshape(dwt_output, [batch_size, -1]),
                                         tf.reshape(lvl0_output, [batch_size, -1]),
                                         tf.reshape(lvl1_output, [batch_size, -1]),
                                         tf.reshape(lvl2_output, [batch_size, -1]),
                                         tf.reshape(lvl3_output, [batch_size, -1])], axis=-1)

        # Commenting out the original self-attention layer
        # attention_output = self.self_attention(concatenated_output)

        # Using the combined dense layers
        final_output = self.final_nn(concatenated_output)

        if self.debug:
            print(dwt_output.shape)
            print(lvl0_output.shape)
            print(lvl1_output.shape)
            print(lvl2_output.shape)
            print(lvl3_output.shape)
            print(concatenated_output.shape)
            print(final_output.shape)

        return final_output

    def train_model(self, dataset, epochs=100, batch_size=64, lr=0.01):
        formatted_dataset = format_dataset(dataset)
        train_dataset = formatted_dataset.batch(batch_size)

        loss = None
        t0 = time.time()
        n = int(len(dataset) // batch_size) + 1

        criterion = CategoricalCrossentropy(from_logits=False)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=lr,
            decay_steps=10000,
            decay_rate=0.9)
        optimizer = Adam(learning_rate=lr_schedule, clipnorm=1.0)

        checkpoint_filepath = os.path.join(self.folder, 'checkpoint.chk')
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='loss',
            mode='min',
            save_best_only=True)

        for epoch in range(epochs):
            print(f'Starting epoch {epoch + 1}/{epochs}...')
            t1 = time.time()

            for batch_idx, (data, target) in enumerate(train_dataset):
                target = tf.convert_to_tensor([tf.one_hot(t, len(self.possible_labels)) for t in target],
                                              dtype=tf.float32)

                with tf.GradientTape() as tape:
                    output = self(data, training=True)
                    loss = criterion(target, output)
                gradients = tape.gradient(loss, self.trainable_variables)
                optimizer.apply_gradients(zip(gradients, self.trainable_variables))

                print_progress_bar('Batches processed', batch_idx + 1, n, start_time=t1)

            if loss is not None:
                elps = int(time.time() - t0)
                elps_m, elps_s = divmod(int(elps), 60)
                print(f'\nEpoch {epoch + 1} completed, Loss: {loss.numpy()}, Time elapsed: {elps_m}:{elps_s}')

            # Save the model at the end of each epoch
            self.save_weights(checkpoint_filepath)

    def predict(self, inputs):
        if len(inputs[0].shape) != 4 or inputs[0].shape[0] != 1:
            raise ValueError("Input batch size should be 1")

        output = self(inputs, training=False)
        probabilities = tf.nn.softmax(output)
        max_index = tf.argmax(probabilities, axis=1)
        max_index_int = int(max_index[0].numpy())
        label_prob_dict = {label: prob.numpy() for label, prob in zip(self.possible_labels, probabilities[0])}

        return str(self.possible_labels[max_index_int]), label_prob_dict

In [5]:
# data preprocessing functions
def process_entry(entry):
    """Process the entropy results to extract the levels."""
    label = entry['label']
    machine_input = {0: [], 1: [], 2: [], 3: [], 'dwt': []}
    for ent in entry['entropy_results']:

        if ent['method'] == 'dwt':
            machine_input['dwt'] = tf.convert_to_tensor(ent['result'], dtype=tf.float32)
        else:
            for lvl, content in enumerate(ent['result']):
                machine_input[lvl].append(tf.convert_to_tensor(content, dtype=tf.float32))

    machine_input[0] = tf.concat(machine_input[0], axis=-1)
    machine_input[1] = tf.concat(machine_input[1], axis=-1)
    machine_input[2] = tf.concat(machine_input[2], axis=-1)
    machine_input[3] = tf.concat(machine_input[3], axis=-1)
    machine_input['dwt'] = tf.reshape(machine_input['dwt'], [1, 1, 10])

    return {'input': machine_input, 'label': label}


def format_dataset(dataset):
    """Formats and shuffles the dataset for training"""
    label_num = {'nature': 0, 'country': 1, 'city': 2}
    formatted_dataset = []
    for item in dataset:
        machine_input = item['input']
        label = label_num[item['label']]
        formatted_dataset.append((
            (
                machine_input[0],
                machine_input[1],
                machine_input[2],
                machine_input[3],
                machine_input['dwt']
            ),
            label
        ))
    return tf.data.Dataset.from_generator(
        lambda: iter(formatted_dataset),
        output_signature=(
            (
                tf.TensorSpec(shape=(1, 1, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(2, 2, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(4, 4, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(8, 8, 17), dtype=tf.float32),
                tf.TensorSpec(shape=(1, 1, 10), dtype=tf.float32)
            ),
            tf.TensorSpec(shape=(), dtype=tf.int32),
        )
    ).shuffle(buffer_size=len(dataset))


def process_json(path, test_part, parallel_jobs=4):
    """Process JSON data to extract dataset and features."""
    with open(path, 'r') as f:
        metadata = json.load(f)
    dataset = []

    t = time.time()
    n = len(metadata)

    with ThreadPoolExecutor(max_workers=parallel_jobs) as executor:
        futures = [executor.submit(process_entry, entry) for entry in metadata]
        for i, future in enumerate(as_completed(futures)):
            result = future.result()
            if result is not None:
                dataset.append(result)
            print_progress_bar('Processed entry', i + 1, n, t)

    if isinstance(test_part, float):
        i = int(test_part * len(dataset))
    elif isinstance(test_part, str):
        i = int(test_part)
    else:
        raise ValueError("Incompatible format for 'test_part'.")

    test_set = dataset[-i:]
    dataset = dataset[:-i]

    num_classes = len(all_labels)
    dataset_length = len(dataset)

    return dataset, test_set, num_classes, dataset_length

In [6]:
# model evaluation functions
def calculate_stats(y_true, y_pred, y_prob):
    """
    This function calculates various statistics like confusion matrix, precision, recall, F1 score and log loss.
    
    Parameters:
    y_true (numpy array): Array of true labels
    y_pred (numpy array): Array of predicted labels
    y_prob (numpy array): Array of predicted probabilities
    
    Returns:
    dict: A dictionary containing all the calculated statistics
    """
    lb = LabelBinarizer()
    lb.fit(y_true)
    y_true_bin = lb.transform(y_true)

    y_prob = np.array([list(item.values()) for item in y_prob])

    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    logloss = log_loss(y_true_bin, y_prob, labels=lb.classes_)

    stats = {
        'confusion_matrix': conf_matrix,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'log_loss': logloss
    }

    return stats


def evaluate_model(model, test_set):
    stats = {'test_samples': 0, 'right_predictions': 0}
    y_true = []
    y_pred = []
    y_prob = []

    for test in test_set:
        stats['test_samples'] += 1

        machine_input = test['input']
        inputs = (
            tf.expand_dims(machine_input[0], axis=0),
            tf.expand_dims(machine_input[1], axis=0),
            tf.expand_dims(machine_input[2], axis=0),
            tf.expand_dims(machine_input[3], axis=0),
            tf.expand_dims(machine_input['dwt'], axis=0)
        )
        true_label = test['label']

        predicted_label, label_probs = model.predict(inputs)

        y_true.append(true_label)
        y_pred.append(predicted_label)
        y_prob.append(label_probs)

        if predicted_label == true_label:
            stats['right_predictions'] += 1

    stats['success_rate'] = 100 * stats['right_predictions'] / stats['test_samples']

    # # Get the calculated stats
    # stats.update(calculate_stats(np.array(y_true), np.array(y_pred), np.array(y_prob)))

    print(f"{stats['right_predictions']} samples out of {stats['test_samples']} were predicted correctly.\n"
          f"The model's success rate is: {stats['success_rate']}%")
    # print(f"Confusion Matrix: \n{stats['confusion_matrix']}")
    # print(f"Precision: {stats['precision']}")
    # print(f"Recall: {stats['recall']}")
    # print(f"F1 Score: {stats['f1_score']}")
    # print(f"Log Loss: {stats['log_loss']}")

    return stats['success_rate'] / 100

In [7]:
# load data
dataset, test_set, num_classes, dataset_length = process_json(path, test_part)
print('\nDataset processed.')
print(f"Total number of entries in the dataset: {dataset_length}")
print(f"Total number of entries in the test set: {len(test_set)}")
print(f"Number of classes: {num_classes}")

Processed entry: ██████████████████████████████████████████████████ | Completed: 7495/7495, 100.0% | Time elapsed: 0:00:22/0:00:22 | Time left: ~= 0:00:00 |
Dataset processed.
Total number of entries in the dataset: 6746
Total number of entries in the test set: 749
Number of classes: 3


In [29]:
# model creation
model_folder = f"../models/EntropyClassifier"
model_name = f"instance_e={epochs}_ds={dataset_length}_bs={batch_size}"
model = EntropyClassifier(all_labels, model_folder)
# model.debug = True
print('Model created')

Model created


In [ ]:
# train model
# model.debug = False
model.train_model(dataset, epochs=epochs, batch_size=batch_size, lr=learning_rate)
print('Model trained.')

Starting epoch 1/20...
Batches processed: ██████████████████████████████████████████████████ | Completed: 27/27, 100.0% | Time elapsed: 0:00:14/0:00:14 | Time left: ~= 0:00:00 |
Epoch 1 completed, Loss: 0.7871942520141602, Time elapsed: 0:14
Starting epoch 2/20...
Batches processed: ██████████████████████████████████████████████████ | Completed: 27/27, 100.0% | Time elapsed: 0:00:11/0:00:11 | Time left: ~= 0:00:00 |
Epoch 2 completed, Loss: 0.7537193298339844, Time elapsed: 0:26
Starting epoch 3/20...
Batches processed: ██████████████████████████████████████████████████ | Completed: 27/27, 100.0% | Time elapsed: 0:00:11/0:00:11 | Time left: ~= 0:00:00 |
Epoch 3 completed, Loss: 0.6115423440933228, Time elapsed: 0:38
Starting epoch 4/20...
Batches processed: ██████████████████████████████████████████████████ | Completed: 27/27, 100.0% | Time elapsed: 0:00:10/0:00:10 | Time left: ~= 0:00:00 |
Epoch 4 completed, Loss: 0.7252815961837769, Time elapsed: 0:49
Starting epoch 5/20...
Batches p

In [15]:
# save model
if not os.path.exists(model_folder):
    os.mkdir(model_folder)
model.save_weights(os.path.join(model_folder, model_name))
print("Model saved")

Model saved


In [34]:
# load model
model.save_weights(os.path.join(model_folder, model_name))
print("model loaded")

model loaded


In [27]:
# model evaluation
comment = 'new architecture. less layers.'
result = evaluate_model(model, test_set)

perf_json = f"{model_folder}/performance.json"
if os.path.exists(perf_json):
    with open(perf_json, 'r') as f:
        perf = json.load(f)
else:
    perf = []

perf.append({'lr': learning_rate,
             'epochs': epochs,
             'dataset size': dataset_length,
             'testset size': len(test_set),
             'test part': test_part,
             'batch size': batch_size,
             'performance': result,
             'comment': comment
             })
with open(perf_json, 'w') as f:
    json.dump(perf, f, indent=4)

361 samples out of 749 were predicted correctly.
The model's success rate is: 48.197596795727634%
